In [1]:
import re
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Cleaning data

In [2]:
# Title: Chỉ xét phòng trọ: ở ghép hoặc không ở ghép
def title_clean(feature, df):
    df[feature] = df[feature].str.lower()
    index_lst = []
    e = 'phòng trọ'
    for i in range(df.shape[0]):
        if re.findall(e, df.Title[i]) == []:
            index_lst.append(i)
    df = df.drop(index_lst).reset_index(drop = True)
    return df


In [3]:
df = pd.read_csv('clean_data.csv')
df = title_clean('Title', df)

Xét phòng trọ có nội thất không? Và ở chung hay ở ghép

In [4]:
def isNew(feature, df):
    df[feature] = df[feature].str.lower()
    e = ['mới', 'mới nhất']
    # df['isNew'] = df[feature].str.contains(e)
    df['isNew'] = df[feature].apply(lambda x: 'yes' if any(i in x for i in e) else 'no')
    return df

In [5]:
def hasFurniture(feature, df):
    df[feature] = df[feature].str.lower()
    e = ['nội thất', 'cao cấp']
    e2 = ['tiện nghi', 'đầy đủ'] # sample 28
    df['hasFurniture'] = df[feature].apply(lambda x: 'yes' if any(i in x for i in e) or all(i in x for i in e2) else 'no')
    return df

In [6]:
# Có ở ghép không?
def liveTogether(feature, df):
    df[feature] = df[feature].str.lower()
    e = ['ghép', 'share']
    df['liveTogether'] = df[feature].apply(lambda x: 'yes' if any(i in x for i in e) else 'no')
    return df

Lúc thu thập dữ liệu, có những chỗ 3 triệu 5 thì họ ghi 3.5, có chỗ ghi 3.500.000, nên cần xử lý.


In [7]:
def cleanPrice(feature, df):
    df[feature] = np.where((df[feature] > 1000) & (df[feature] < 10000), df[feature]/1000, df[feature])
    df[feature] = np.where((df[feature] < 1000) & (df[feature] > 100), df[feature]/100, df[feature])
    df[feature] = np.where((df[feature] > 1000000),df[feature]/1000000, df[feature])    

    # Xóa bỏ những dòng mà Price bằng null
    df = df[df["Price"].notna()]
    return df

df = cleanPrice('Price', df)

Univariate Analysis 

In [8]:
def univariate_cont_analysis(var,df):
    mean = df[var].mean()
    median = df[var].median() 
    mode = df[var].mode()
    max_val = df[var].max()
    min_val = df[var].min()
    range_val = np.ptp(df[var])
    variance = df[var].var()
    skewness = df[var].skew()
    result = [mean,median,mode,max_val,min_val,range_val,variance]
    return result

In [9]:
#Visualization - Histogram
def visualize_histogram(cont_var, df):
    if len(cont_var)%3 == 0:
        n = len(cont_var)/3
    else:
        n = len(cont_var)//3 + 1
    plt.figure(figsize=(15,15))
    for i in range(1,len(cont_var)+1):
        plt.subplot(n,3,i)
        sns.distplot(df[cont_var[i-1]].dropna())
    plt.tight_layout()
    plt.show()